In [32]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

df1 = pd.read_csv('/Volumes/imvDrive/cfdb-django/glyphs-aligned-w-std_sign-images.csv', usecols=['std_sign', 'glyph_img'])
df2 = pd.read_csv('/Volumes/imvDrive/cfdb-django/glyph_count.csv', usecols=['std_sign', 'glyph']).sort_values(by='glyph')

start_file = '/Volumes/imvDrive/cfdb-django/media/start_file.csv'

if start_file in os.listdir('/Volumes/imvDrive/cfdb-django/media/'):
    os.remove(start_file)

f=open(start_file, 'a+')

df_group = df1.groupby(by=['std_sign'])
df_group = sorted(df_group, key=lambda x: len(x[1])) #https://stackoverflow.com/questions/22291395/sorting-the-grouped-data-as-per-group-size-in-pandas

for name,group in df_group:
    if  group.count()[0] >= 50:
        if(os.stat(start_file).st_size == 0):
            group.reset_index().to_csv(f)
        else:
            group.reset_index().to_csv(f, header=None)
        
df_usable = pd.read_csv(start_file)
df_usable = df_usable.drop(['Unnamed: 0', 'index'], axis=1)
df_usable.to_csv('/Volumes/imvDrive/cfdb-django/media/start_file.csv')

'''
glyph_list = []
sign_list = []

## too computationally expensive to amplify list of data
## keep basic list and call _e, _s, _w pictures during model training

for index, row in df_usable.iterrows():
    if '.' in row['glyph_img']:
        split_glyph = row['glyph_img'].split('.')
        glyph_e = split_glyph[0]+'_e.'+split_glyph[1] 
        glyph_s = split_glyph[0]+'_s.'+split_glyph[1] 
        glyph_w = split_glyph[0]+'_w.'+split_glyph[1] 
        glyph_list.append(glyph_e)
        glyph_list.append(glyph_s)
        glyph_list.append(glyph_w)

        split_sign = row['std_sign'].split('.')
        sign_e = split_sign[0]+'_e.'+split_sign[1] 
        sign_s = split_sign[0]+'_s.'+split_sign[1] 
        sign_w = split_sign[0]+'_w.'+split_sign[1] 
        sign_list.append(sign_e)
        sign_list.append(sign_s)
        sign_list.append(sign_w)
print(glyph_list)
print(sign_list)
'''

### TRAINING, VALIDATION, TESTING SETS
### can be more efficiently done in tensorflow
### see below

'''
df_shuffled = df_usable.sample(frac=1).reset_index(drop=True)
df_shuffled = df_shuffled.drop(['Unnamed: 0', 'index'], axis=1)

split_index0 = int(df_shuffled.count()[0]*0.75)
df_pre_testing = df_shuffled.iloc[:split_index0, :]
df_testing = df_shuffled.iloc[:split_index0, :]
df_testing.to_csv('/Volumes/imvDrive/cfdb-django/media/test_set/testing_data.csv')

split_index1 = int(df_pre_testing.count()[0]*0.95)
df_training = df_pre_testing.iloc[:split_index1, :]
df_validation = df_pre_testing.iloc[split_index1:, :]

df_training.to_csv('/Volumes/imvDrive/cfdb-django/media/training_set/training_data.csv')
df_validation.to_csv('/Volumes/imvDrive/cfdb-django/media/validation_set/validation_data.csv')
'''

"\ndf_shuffled = df_usable.sample(frac=1).reset_index(drop=True)\ndf_shuffled = df_shuffled.drop(['Unnamed: 0', 'index'], axis=1)\n\nsplit_index0 = int(df_shuffled.count()[0]*0.75)\ndf_pre_testing = df_shuffled.iloc[:split_index0, :]\ndf_testing = df_shuffled.iloc[:split_index0, :]\ndf_testing.to_csv('/Volumes/imvDrive/cfdb-django/media/test_set/testing_data.csv')\n\nsplit_index1 = int(df_pre_testing.count()[0]*0.95)\ndf_training = df_pre_testing.iloc[:split_index1, :]\ndf_validation = df_pre_testing.iloc[split_index1:, :]\n\ndf_training.to_csv('/Volumes/imvDrive/cfdb-django/media/training_set/training_data.csv')\ndf_validation.to_csv('/Volumes/imvDrive/cfdb-django/media/validation_set/validation_data.csv')\n"

In [45]:
from random import shuffle
import glob
import sys
import numpy as np
from PIL import Image

# DATASTRUCTURES
### testing upload
### source: Daniel Persson,'How to load a custom dataset with tf.data [Tensorflow]', https://www.youtube.com/watch?v=bqeUmLCgsVw
    
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.trainBytesList(value=[value]))

'''
def load_image(addr):
    img = cv2.imread(addr)
    if img is None:
        return None 
    img = cv2.resize(img,(224, 224), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img
'''

def createDataRecord(outFileName, addrs, labels):
    print('Creating data record...')
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(outFileName)
    for i in range(len(addrs)):
        # print how many images are saved every 1000 images
        if not i % 1000:
            print('Start data: {}/{}'.format(i,len(addrs)))
            sys.stdout.flush()
        # load the image
        ## img = cv2.imread(glyphtosignlist.iloc[i,0])
        filename = os.fsdecode('/Volumes/imvDrive/cfdb-django/media/glyph_img/'+addrs.iloc[i])
        try:
            img = Image.open(filename).convert('L') #convert to grayscale
            img = img.resize((331,331)) ###uniform resize
            label = labels.iloc[i]
    
            # create a feature
            feature = {
                'glyph_img_raw': _bytes_feature(img.tostring()),
                'label_img_raw': _int64_feature(label)
            }
            
             # create an example protocol buffer
            example = tf.train.Example(features=tf.train.Features(feature=feature))

            # serialize to string and write on the file
            writer.write(example.SerializeToString())
        except Exception:
            pass
        
    writer.close()
    sys.stdout.flush()
    
glyphtosignpath = pd.read_csv("/Volumes/imvDrive/cfdb-django/media/start_file.csv")
# read addresses and labels from the 'train' folder
# print(str(glyphtosignpath.iloc[:,1]))
addrs = glyphtosignpath.iloc[:,1]
labels = glyphtosignpath.iloc[:,2]
    
'''
# to shuffle data
c = list(zip(addrs, labels))
shuffle(c)
addrs, labels = zip(*c)
'''

# divide the data into 80% train, 10% validation, and 10% test
train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]
val_addrs = addrs[int(0.8*len(addrs)):int(0.9*len(addrs))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
test_addrs = addrs[int(0.9*len(addrs)):]
test_labels = labels[int(0.9*len(labels)):]

    
createDataRecord('train.tfrecords', train_addrs, train_labels)
createDataRecord('val.tfrecords', val_addrs, val_labels)
createDataRecord('test.tfrecords', test_addrs, test_labels)

print("DONE")

sess = tf.Session()
sess.run(tf.global_variables_initializer())

def parser(record):
    keys_to_features = {
        "image_raw": tf.FixedLenFeature([], tf.string),
        "label": tf.FixedLenFeature([], tf.int64)
    }
    
    parsed = tf.parse_single_example(record, keys_to_features)
    image = tf.decode_raw(parsed["image_raw"], tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshaped(image, shape=[331, 331,2])
    label = tf.cast(parsed["label"], tf.int32)
    
    return image, label

def input_fn(filenames, train, batch_size=32, buffer_size=2048):
    dataset = tf.data.TFRRecordDataset(filenames=filenames)
    dataset = dataset.map(parser)
    
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
        num_repeat = None
    else:
        num_repeat = 1
        
    dataset = dataset.repeat(num_repeat)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_interator()
    images_batch, labels_batch = iterator.get_next()
    
    x = {'image': images_batch}
    y = labels_batch
    
    return x, y

def train_input_fn():
    return input_fn(filenames=["train.tfrecords", "test.tfrecords"], train=True)

def val_input_fn():
    return input_fn(filenames=["val.tfrecords"], train=False)


Creating data record...
Start data: 0/9538
Start data: 1000/9538
Start data: 2000/9538
Start data: 3000/9538
Start data: 4000/9538
Start data: 5000/9538
Start data: 6000/9538
Start data: 7000/9538
Start data: 8000/9538
Start data: 9000/9538
Creating data record...
Start data: 0/1192
Start data: 1000/1192
Creating data record...
Start data: 0/1193
Start data: 1000/1193
DONE
